In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
from IPython import display
import time
import os

# proxyの設定．
# keras.datasetsでは，datasetを直接ダウンロードするので，学内マシンからは通常必要．
os.environ["http_proxy"] = "http://proxy.uec.ac.jp:8080/"
os.environ["https_proxy"] = "http://proxy.uec.ac.jp:8080/"
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"      # "0":GPU0, "1":GPU1, "0,1":GPUを2つとも使用

import numpy as np
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from skimage import feature,filters
from PIL import Image
import sys

import cv2

# ResNet50 による 1000種類分類
import torchvision.models as models
import pickle
import glob


In [ ]:
img = cv2.imread('images/noguchi01.jpg')
# show image with matplotlib
plt.imshow(img)

In [2]:

LIST=['pasta/','ramen/','somen/','soba/','udon/']
DIR0='./img/'
dir_end='*.jpg'

img_list = []

for dir_pos in LIST:
    print(glob.glob( DIR0+dir_pos+dir_end ))
    img_list.extend(glob.glob( DIR0+dir_pos+dir_end ))

['./img/pasta/000002.jpg', './img/pasta/000001.jpg', './img/pasta/000006.jpg', './img/pasta/000007.jpg', './img/pasta/000008.jpg', './img/pasta/000010.jpg', './img/pasta/000012.jpg', './img/pasta/000014.jpg', './img/pasta/000015.jpg', './img/pasta/000016.jpg', './img/pasta/000022.jpg', './img/pasta/000023.jpg', './img/pasta/000024.jpg', './img/pasta/000025.jpg', './img/pasta/000028.jpg', './img/pasta/000029.jpg', './img/pasta/000032.jpg', './img/pasta/000036.jpg', './img/pasta/000040.jpg', './img/pasta/000041.jpg', './img/pasta/000042.jpg', './img/pasta/000044.jpg', './img/pasta/000047.jpg', './img/pasta/000049.jpg', './img/pasta/000051.jpg', './img/pasta/000052.jpg', './img/pasta/000053.jpg', './img/pasta/000056.jpg', './img/pasta/000057.jpg', './img/pasta/000059.jpg', './img/pasta/000060.jpg', './img/pasta/000061.jpg', './img/pasta/000062.jpg', './img/pasta/000064.jpg', './img/pasta/000065.jpg', './img/pasta/000068.jpg', './img/pasta/000069.jpg', './img/pasta/000071.jpg', './img/past

In [20]:
img_list[0]

'./img/pasta/000002.jpg'

In [3]:
#pickled_data.decode('utf8')
f = open('list.txt','wb')
pickle.dump(img_list,f)

NameError: name 'pickled_data' is not defined

In [2]:
f = open("list.txt", "rb")
imgs_list = pickle.load(f)

In [46]:
len(imgs_list)

442

In [58]:
c_hist = np.empty((512, ), dtype='float32')
for dir_pos in imgs_list:
    img = cv2.imread(dir_pos)
    hist_c = cv2.calcHist([img], [0, 1, 2], None, [8, 8, 8],[0, 256, 0, 256, 0, 256])
    hist_c = hist_c.flatten()
    hist_c = hist_c / sum(hist_c)
    #print(sum(hist_c))
    #print(hist_c.dtype)
    c_hist = np.vstack((c_hist, hist_c))
    #break

0.9999999992746211
1.0000000008021743
1.0000000018535502
0.999999998045837
0.999999990213837
1.0000000058848855
1.0000000071688078
0.9999999958766352
1.0000000084953626
1.0000000042317652
1.0000000024572273
1.000000002181423
0.9999999865945597
0.999999997550276
1.0000000031814125
1.0000000007894414
0.9999999979949052
0.9999999999650981
0.999999996056431
1.0000000013687895
0.9999999994508926
0.9999999971496436
1.0000000018553692
1.000000002723823
0.9999999998909743
1.0
1.000000000606633
0.9999999984411261
0.9999999967020585
1.0000000008149073
1.0000000216040235
1.0000000047530193
0.9999999958566264
1.0000000070269834
0.9999999974019147
1.000000006636668
1.0000000018013964
1.0000000039112251
0.9999999811350442
1.000000006636668
1.0000000044274202
1.0000000070269834
1.0000000035383039
0.9999999965093593
1.0000000018013964
0.9999999931533239
1.0000000039112251
1.0000000019837216
1.0000000022603217
1.0000000011075372
0.9999999945268314
0.9999999998237854
0.9999999984859187
0.999999996726955

In [59]:
#print(c_hist[1])
print(c_hist.shape)
c_hist = np.delete(c_hist, 0, 0)
print(c_hist.shape)

[2.15412751e-02 1.30627230e-02 2.50355024e-02 9.30156745e-03
 9.43574880e-04 3.08749848e-04 8.94944387e-05 5.97426151e-07
 7.44392979e-04 1.16044062e-03 8.50914046e-03 3.75205129e-02
 4.75957477e-03 2.11727835e-04 7.49172395e-05 2.74816034e-06
 1.34062429e-04 5.90137555e-04 7.66975689e-04 1.71586778e-02
 4.21794839e-02 4.05042991e-03 1.98464972e-04 7.28859914e-06
 0.00000000e+00 8.66267947e-05 5.06497920e-04 4.58225870e-04
 2.05489509e-02 4.05816063e-02 2.11691996e-03 1.48161689e-05
 0.00000000e+00 0.00000000e+00 9.76194351e-05 3.00744345e-04
 3.03970446e-04 1.45194875e-02 1.18080089e-02 8.88970171e-05
 0.00000000e+00 0.00000000e+00 0.00000000e+00 1.29044056e-05
 9.22425970e-05 1.95119384e-04 1.45724195e-03 1.33345515e-04
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 1.19485236e-07 3.10661608e-06 1.43382283e-06 3.58455708e-07
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 6.40440849e-05 2.628675

In [60]:
f = open('c_hist.txt','wb')
pickle.dump(c_hist,f)

In [61]:
f = open("c_hist.txt", "rb")
c_hists = pickle.load(f)
c_hists == c_hist

array([[ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       ...,
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True]])

In [70]:
d_L1 = cv2.compareHist(c_hist[0], c_hist[1], cv2.HISTCMP_INTERSECT)
print(d_L1)
d_L2 = (sum((c_hist[0] - c_hist[3])**2)**(0.5))
print(d_L2)

0.39092010874773564
0.2833610711625657


In [75]:
hsv_hist = np.empty((512, ), dtype='float32')
for dir_pos in imgs_list:
    img = cv2.imread(dir_pos)
    img = cv2.cvtColor(img ,cv2.COLOR_BGR2HSV)
    hist_hsv = cv2.calcHist([img], [0, 1, 2], None, [8, 8, 8],[0, 256, 0, 256, 0, 256])
    hist_hsv = hist_hsv.flatten()
    hist_hsv = hist_hsv / sum(hist_hsv)
    hsv_hist = np.vstack((hsv_hist, hist_hsv))
    #break
#print(c_hist[1])
print(hsv_hist.shape)
hsv_hist = np.delete(hsv_hist, 0, 0)
print(hsv_hist.shape)
f = open('hsv_hist.txt','wb')
pickle.dump(hsv_hist,f)

(443, 512)
(442, 512)


float32


In [76]:
f = open("hsv_hist.txt", "rb")
hsv_hists = pickle.load(f)
hsv_hists == hsv_hist

array([[ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       ...,
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True]])

In [77]:
luv_hist = np.empty((512, ), dtype='float32')
for dir_pos in imgs_list:
    img = cv2.imread(dir_pos)
    img = cv2.cvtColor(img ,cv2.COLOR_BGR2Luv)
    hist_luv = cv2.calcHist([img], [0, 1, 2], None, [8, 8, 8],[0, 256, 0, 256, 0, 256])
    hist_luv = hist_luv.flatten()
    hist_luv = hist_luv / sum(hist_luv)
    luv_hist = np.vstack((luv_hist, hist_luv))
    #break
#print(c_hist[1])
print(luv_hist.shape)
luv_hist = np.delete(luv_hist, 0, 0)
print(luv_hist.shape)
f = open('luv_hist.txt','wb')
pickle.dump(luv_hist,f)

(443, 512)
(442, 512)


In [104]:
split = 2
c2x2_hist = np.empty((512*split**2, ), dtype='float32')
for dir_pos in imgs_list:
    img = cv2.imread(dir_pos)
    split_hist = np.empty((512, ), dtype='float32')
    #img split
    chunks = []
    for row_img in np.array_split(img, split, axis=0):
        for chunk in np.array_split(row_img, split, axis=1):
            chunks.append(chunk)
    print(len(chunks))
    print(chunks[0].shape)
    #img -> hist + flat
    for i in range(split**2):
        hist_c = cv2.calcHist([chunks[0]], [0, 1, 2], None, [8, 8, 8],[0, 256, 0, 256, 0, 256])
        hist_c = hist_c.flatten()
        split_hist = np.vstack((split_hist, hist_c))
    
    split_hist = np.delete(split_hist, 0, 0)
    split_hist = split_hist.flatten()
    split_hist = split_hist / sum(split_hist)
    print(split_hist.shape)
    c2x2_hist = np.vstack((c2x2_hist, split_hist))
    #break
c2x2_hist = np.delete(c2x2_hist, 0, 0)
print(c2x2_hist.shape)

4
(1712, 1223, 3)
(2048,)
4
(200, 220, 3)
(2048,)
4
(250, 250, 3)
(2048,)
4
(539, 300, 3)
(2048,)
4
(250, 250, 3)
(2048,)
4
(325, 325, 3)
(2048,)
4
(563, 750, 3)
(2048,)
4
(768, 512, 3)
(2048,)
4
(267, 400, 3)
(2048,)
4
(900, 600, 3)
(2048,)
4
(1044, 696, 3)
(2048,)
4
(450, 450, 3)
(2048,)
4
(959, 640, 3)
(2048,)
4
(175, 350, 3)
(2048,)
4
(188, 250, 3)
(2048,)
4
(300, 300, 3)
(2048,)
4
(750, 585, 3)
(2048,)
4
(660, 990, 3)
(2048,)
4
(320, 384, 3)
(2048,)
4
(240, 320, 3)
(2048,)
4
(552, 368, 3)
(2048,)
4
(340, 340, 3)
(2048,)
4
(320, 384, 3)
(2048,)
4
(480, 384, 3)
(2048,)
4
(442, 600, 3)
(2048,)
4
(1, 1, 3)
(2048,)
4
(480, 320, 3)
(2048,)
4
(600, 600, 3)
(2048,)
4
(473, 600, 3)
(2048,)
4
(323, 323, 3)
(2048,)
4
(1266, 1583, 3)
(2048,)
4
(310, 350, 3)
(2048,)
4
(1092, 728, 3)
(2048,)
4
(534, 800, 3)
(2048,)
4
(720, 720, 3)
(2048,)
4
(900, 600, 3)
(2048,)
4
(1020, 680, 3)
(2048,)
4
(563, 500, 3)
(2048,)
4
(2406, 1805, 3)
(2048,)
4
(900, 600, 3)
(2048,)
4
(188, 250, 3)
(2048,)
4
(534, 800

4
(750, 1000, 3)
(2048,)
4
(1025, 1025, 3)
(2048,)
4
(512, 341, 3)
(2048,)
4
(315, 600, 3)
(2048,)
4
(683, 1024, 3)
(2048,)
4
(484, 323, 3)
(2048,)
4
(268, 375, 3)
(2048,)
4
(750, 1000, 3)
(2048,)
4
(360, 640, 3)
(2048,)
4
(200, 250, 3)
(2048,)
4
(484, 323, 3)
(2048,)
4
(315, 600, 3)
(2048,)
4
(1424, 2136, 3)
(2048,)
4
(250, 375, 3)
(2048,)
4
(433, 650, 3)
(2048,)
4
(450, 600, 3)
(2048,)
4
(800, 600, 3)
(2048,)
4
(266, 376, 3)
(2048,)
4
(1080, 720, 3)
(2048,)
4
(266, 376, 3)
(2048,)
4
(667, 1000, 3)
(2048,)
4
(401, 401, 3)
(2048,)
4
(120, 180, 3)
(2048,)
4
(338, 512, 3)
(2048,)
4
(360, 640, 3)
(2048,)
4
(360, 540, 3)
(2048,)
4
(200, 220, 3)
(2048,)
4
(165, 320, 3)
(2048,)
4
(750, 1000, 3)
(2048,)
4
(630, 1120, 3)
(2048,)
4
(250, 250, 3)
(2048,)
4
(200, 300, 3)
(2048,)
4
(180, 200, 3)
(2048,)
4
(427, 640, 3)
(2048,)
4
(1324, 1655, 3)
(2048,)
4
(238, 229, 3)
(2048,)
4
(200, 220, 3)
(2048,)
4
(480, 640, 3)
(2048,)
4
(540, 960, 3)
(2048,)
4
(360, 640, 3)
(2048,)
4
(318, 478, 3)
(2048,)
4
(

In [105]:
f = open('c2x2_hist.txt','wb')
pickle.dump(c2x2_hist,f)

In [6]:
def split_img(img, split):
    chunks = []
    for row_img in np.array_split(img, split, axis=0):
        for chunk in np.array_split(row_img, split, axis=1):
            chunks.append(chunk)
    return chunks

In [7]:
split = 3
c3x3_hist = np.empty((512*split**2, ), dtype='float32')
for dir_pos in imgs_list:
    img = cv2.imread(dir_pos)
    split_hist = np.empty((512, ), dtype='float32')
    #img split
    chunks = split_img(img, split)
    #img -> hist + flat
    for i in range(split**2):
        hist_c = cv2.calcHist([chunks[0]], [0, 1, 2], None, [8, 8, 8],[0, 256, 0, 256, 0, 256])
        hist_c = hist_c.flatten()
        split_hist = np.vstack((split_hist, hist_c))
    
    split_hist = np.delete(split_hist, 0, 0)
    split_hist = split_hist.flatten()
    split_hist = split_hist / sum(split_hist)
    #print(split_hist.shape)
    c3x3_hist = np.vstack((c3x3_hist, split_hist))
    #break
c3x3_hist = np.delete(c3x3_hist, 0, 0)
print(c3x3_hist.shape)

(442, 4608)


In [9]:
f = open('c3x3_hist.txt','wb')
pickle.dump(c3x3_hist,f)

In [10]:
split = 2
hsv2x2_hist = np.empty((512*split**2, ), dtype='float32')
for dir_pos in imgs_list:
    img = cv2.imread(dir_pos)
    img = cv2.cvtColor(img ,cv2.COLOR_BGR2HSV)
    split_hist = np.empty((512, ), dtype='float32')
    #img split
    chunks = split_img(img, split)
    #img -> hist + flat
    for i in range(split**2):
        hist_c = cv2.calcHist([chunks[0]], [0, 1, 2], None, [8, 8, 8],[0, 256, 0, 256, 0, 256])
        hist_c = hist_c.flatten()
        split_hist = np.vstack((split_hist, hist_c))
    
    split_hist = np.delete(split_hist, 0, 0)
    split_hist = split_hist.flatten()
    split_hist = split_hist / sum(split_hist)
    #print(split_hist.shape)
    hsv2x2_hist = np.vstack((hsv2x2_hist, split_hist))
    #break
hsv2x2_hist = np.delete(hsv2x2_hist, 0, 0)
print(hsv2x2_hist.shape)

(442, 2048)


In [11]:
f = open('hsv2x2_hist.txt','wb')
pickle.dump(hsv2x2_hist,f)

In [12]:
split = 3
hsv3x3_hist = np.empty((512*split**2, ), dtype='float32')
for dir_pos in imgs_list:
    img = cv2.imread(dir_pos)
    img = cv2.cvtColor(img ,cv2.COLOR_BGR2HSV)
    split_hist = np.empty((512, ), dtype='float32')
    #img split
    chunks = split_img(img, split)
    #img -> hist + flat
    for i in range(split**2):
        hist_c = cv2.calcHist([chunks[0]], [0, 1, 2], None, [8, 8, 8],[0, 256, 0, 256, 0, 256])
        hist_c = hist_c.flatten()
        split_hist = np.vstack((split_hist, hist_c))
    
    split_hist = np.delete(split_hist, 0, 0)
    split_hist = split_hist.flatten()
    split_hist = split_hist / sum(split_hist)
    #print(split_hist.shape)
    hsv3x3_hist = np.vstack((hsv3x3_hist, split_hist))
    #break
hsv3x3_hist = np.delete(hsv3x3_hist, 0, 0)
print(hsv3x3_hist.shape)

(442, 4608)


In [13]:
f = open('hsv3x3_hist.txt','wb')
pickle.dump(hsv3x3_hist,f)

In [14]:
split = 2
luv2x2_hist = np.empty((512*split**2, ), dtype='float32')
for dir_pos in imgs_list:
    img = cv2.imread(dir_pos)
    img = cv2.cvtColor(img ,cv2.COLOR_BGR2Luv)
    split_hist = np.empty((512, ), dtype='float32')
    #img split
    chunks = split_img(img, split)
    #img -> hist + flat
    for i in range(split**2):
        hist_c = cv2.calcHist([chunks[0]], [0, 1, 2], None, [8, 8, 8],[0, 256, 0, 256, 0, 256])
        hist_c = hist_c.flatten()
        split_hist = np.vstack((split_hist, hist_c))
    
    split_hist = np.delete(split_hist, 0, 0)
    split_hist = split_hist.flatten()
    split_hist = split_hist / sum(split_hist)
    #print(split_hist.shape)
    luv2x2_hist = np.vstack((luv2x2_hist, split_hist))
    #break
luv2x2_hist = np.delete(luv2x2_hist, 0, 0)
print(luv2x2_hist.shape)

(442, 2048)


In [15]:
f = open('luv2x2_hist.txt','wb')
pickle.dump(luv2x2_hist,f)

In [16]:
split = 3
luv3x3_hist = np.empty((512*split**2, ), dtype='float32')
for dir_pos in imgs_list:
    img = cv2.imread(dir_pos)
    img = cv2.cvtColor(img ,cv2.COLOR_BGR2Luv)
    split_hist = np.empty((512, ), dtype='float32')
    #img split
    chunks = split_img(img, split)
    #img -> hist + flat
    for i in range(split**2):
        hist_c = cv2.calcHist([chunks[0]], [0, 1, 2], None, [8, 8, 8],[0, 256, 0, 256, 0, 256])
        hist_c = hist_c.flatten()
        split_hist = np.vstack((split_hist, hist_c))
    
    split_hist = np.delete(split_hist, 0, 0)
    split_hist = split_hist.flatten()
    split_hist = split_hist / sum(split_hist)
    #print(split_hist.shape)
    luv3x3_hist = np.vstack((luv3x3_hist, split_hist))
    #break
luv3x3_hist = np.delete(luv3x3_hist, 0, 0)
print(luv3x3_hist.shape)

(442, 4608)


In [17]:
f = open('luv3x3_hist.txt','wb')
pickle.dump(luv3x3_hist,f)

In [3]:
vgg16 = models.vgg16(pretrained=True,progress=True)
softmax=nn.Softmax(dim=1)
# pretrained=True とすると，学習済みポラメータも読み込まれる．
# ~/.cache/torch/checkpoints/ に読み込まれます．VGG16は550MBもあるので，不要になったら消去しましょう．
# ls でダウンロードされていることを確認してみます．
! ls -l ~/.cache/torch/checkpoints/ 

total 540464
-rw------- 1 yamamoto-k YANAI_LAB 553433881 Apr 25 19:43 vgg16-397923af.pth


In [4]:
class Flatten(nn.Module):
    def forward(self, input):
        return input.reshape(input.size(0), -1)
    
vgg16fc7 = torch.nn.Sequential(
    vgg16.features,
    vgg16.avgpool,
    Flatten(),
    *list(vgg16.classifier.children())[:-3]  # 最後の3つのlayer(relu,dropout,fc1000)を削除
)
# 表示してみます．fc7 (fc4096)が最終出力になっているはずです．
#print(vgg16fc7)

In [5]:
# 200枚画像をimgsに読み込みます．
in_size=224
imgs = np.empty((0,in_size,in_size,3), dtype=np.float32)

for i,img_path in enumerate(imgs_list):
    if i%100==0:
        print("reading {}th image".format(i))
    x = np.array(Image.open(img_path).resize((in_size,in_size)), dtype=np.float32)
    if x.shape == (in_size, in_size, 2):
        img_compatible = np.empty((in_size,in_size,1), dtype=np.float32)
        print(img_compatible.shape)
        x = np.dstack([x, img_compatible])
    if x.shape == (in_size, in_size, 4):
        x = x[:,:,0:3]
    x = np.expand_dims(x, axis=0)
    imgs = np.vstack((imgs,x))
    
mean=np.array([0.485, 0.456, 0.406], dtype=np.float32)
std=np.array([0.229, 0.224, 0.225], dtype=np.float32)
imgs=(imgs/255.0-mean)/std
imgs=imgs.transpose(0,3,1,2)  # HWC -> CHW
img=torch.from_numpy(imgs)
print(imgs.shape)

reading 0th image
(224, 224, 1)
reading 100th image
reading 200th image
reading 300th image
reading 400th image
(442, 3, 224, 224)


In [6]:
# 200枚処理するので，GPUを使います．
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
vgg16fc7 = vgg16fc7.to(device)
#if device == 'cuda':
#    vgg16fc7 = torch.nn.DataParallel(vgg16fc7)

cuda


In [ ]:
vgg16fc7.eval()
with torch.no_grad():
    fc=vgg16fc7(img.to(device)).cpu().numpy()
    # gpuで処理した結果を cpuに戻して，numpy形式にします．
print(fc.shape)     # shapeの表示

In [7]:
vgg16fc7.eval()
with torch.no_grad():
    fc_x=vgg16fc7(img[0:100].to(device)).cpu().numpy()
    # gpuで処理した結果を cpuに戻して，numpy形式にします．
#torch.cuda.empty_cache()
#with torch.no_grad():
#    fc_y=vgg16fc7(img[200:].to(device)).cpu().numpy()
    # gpuで処理した結果を cpuに戻して，numpy形式にします．
#fc = np.vstack((fc_x, fc_y))
fc = fc_x
print(fc_x.shape)     # shapeの表示

(100, 4096)


In [10]:
vgg16fc7.eval()
with torch.no_grad():
    fc_x=vgg16fc7(img[300:].to(device)).cpu().numpy()
    # gpuで処理した結果を cpuに戻して，numpy形式にします．
#torch.cuda.empty_cache()
#with torch.no_grad():
#    fc_y=vgg16fc7(img[200:].to(device)).cpu().numpy()
    # gpuで処理した結果を cpuに戻して，numpy形式にします．
#fc = np.vstack((fc_x, fc_y))
fc = np.vstack((fc, fc_x))
print(fc.shape)     # shapeの表示

(442, 4096)


In [11]:
f = open('vgg16_feature.txt','wb')
pickle.dump(fc,f)

In [14]:
f = open("vgg16_feature.txt", "rb")
fc = pickle.load(f)

In [19]:
norm = np.linalg.norm(fc,axis=1)
print(norm.shape)
for i in range(len(norm)):
    fc[i] = fc[i] / norm[i]

(442,)


In [22]:
for i in range(len(norm)):
    print(np.linalg.norm(fc[i]))

0.99999994
0.99999994
1.0000001
1.0
1.0
0.99999994
0.99999994
0.99999994
1.0
0.99999994
1.0
0.99999994
1.0
1.0
0.99999994
1.0
1.0
0.99999994
0.99999994
0.99999994
1.0
1.0
1.0
0.9999999
1.0
0.9999999
1.0
1.0
1.0
1.0
1.0
1.0000001
1.0
1.0
1.0
1.0
1.0
0.99999994
1.0
1.0
0.99999994
1.0
0.99999994
1.0
1.0
1.0
0.99999994
0.99999994
1.0
0.99999994
1.0
0.99999994
1.0
1.0
1.0
0.99999994
1.0
1.0
0.99999994
1.0
1.0
1.0000001
0.99999994
0.99999994
0.99999994
1.0
0.99999994
1.0
0.99999994
0.99999994
1.0
0.99999994
0.99999994
1.0
1.0
0.99999994
0.99999994
0.99999994
0.99999994
1.0000001
1.0000001
0.99999994
1.0
1.0
0.99999994
1.0
1.0
0.99999994
0.99999994
1.0
1.0000001
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.99999994
0.99999994
1.0
1.0
1.0
1.0
1.0
1.0
0.99999994
1.0
1.0
1.0
1.0
1.0
1.0
0.99999994
0.99999994
1.0
0.99999994
0.99999994
1.0
1.0
0.99999994
0.99999994
0.99999994
1.0
0.99999994
0.99999994
1.0
1.0
1.0
0.99999994
1.0
1.0
1.0
0.99999994
1.0
0.99999994
0.99999994
1.0
0.99999994
1.0
1.0
0.99999994
0.9999

In [23]:
f = open('vgg16_feature_norm.txt','wb')
pickle.dump(fc,f)